In [1]:
from sympy import *
from aux_methods import show_expr

In [2]:
# Define symbols to use
k, N, u, r, g = symbols('k, N, mu, r, gamma')

In [3]:
# initial transition always from ground state 
# this is a tuple of size equal to the number of possible mRNA states
# ground = (0,) for the simplest case
# ground = (0, 0) if there are two possible mRNA states
ground = (0, 0, 0)

# an arrival from ground state to first mRNA state
first = (1, 0, 0)
second = (0, 1, 0)
third = (0, 0, 1)

# transition rates between mRNA states
tran_rates = {
    (ground, first): k/N,
    (first, second): 3*u,
    (second, third): 3*u,
    (third, ground): 3*u
}

In [4]:
# will contain the actual protein production rates for all posible
# states of the system
prod_rates = {
    first: r,
    second: r,
    third: r
}

In [5]:
# Determine the set of unique nodes (states)
nodes = set([node for edge in tran_rates.keys() for node in edge])

# Node count, sorted list of nodes
n_cnt = len(nodes)
n_lst = sorted(list(nodes))
n_idx = range(n_cnt)

# dict map from node name to node index in sorted list
n_dct = dict(zip(n_lst, n_idx))

In [6]:
# Build the "transition" matrix
K = zeros(n_cnt, n_cnt)
for t, r in tran_rates.items():
    K[n_dct[t[1]], n_dct[t[0]]] += r
    K[n_dct[t[0]], n_dct[t[0]]] -= r
show_expr(K)

<IPython.core.display.Math object>

In [7]:
R = zeros(n_cnt, n_cnt)
for s, r in prod_rates.items():
    R[n_dct[s], n_dct[s]] += r
show_expr(R)

<IPython.core.display.Math object>

In [8]:
X = K.copy()
X.row_del(0)
X = X.row_insert(0, ones(1, K.shape[0]))
G = eye(K.shape[0])*g
kr = R*ones(K.shape[0], 1)
b = zeros(K.shape[0], 1)
b[0] = 1

In [9]:
m0 = factor(X.LUsolve(b))
m1 = factor((K-G).LUsolve(-R*m0))

In [10]:
# 1st moment at reduced model
mean_rm = factor((kr.T * m0/g)[0])
show_expr(mean_rm)

<IPython.core.display.Math object>

In [11]:
# 1st moment: E[p]
# mean protein number at original model
mean = (N*mean_rm).limit(N, S.Infinity)
show_expr(mean)

<IPython.core.display.Math object>

In [12]:
# 2nd moment at reduced model
secm_rm = factor((kr.T * m1/g)[0])
show_expr(secm_rm)

<IPython.core.display.Math object>

In [13]:
# 2nd moment: E[p*(p-1)]
# for protein number at original model
secm = factor(mean**2 + (N*secm_rm).limit(N, S.Infinity))
show_expr(secm)

<IPython.core.display.Math object>

In [14]:
# compute the fano factor
# for protein number at original model
variance = factor(secm + mean - mean**2)
fano = factor(variance/mean -1) +1 # tweak factorization
show_expr(fano)

<IPython.core.display.Math object>